#Obligaciones de Clientes

Extracción y preparación

In [53]:
#Importación de las Librerias necesarias
import pandas as pd
import numpy as np

In [54]:
#Cargue de la información en Dataframes
#Archivo Obligaciones: Contiene la información de las obligaciones de los clientes con su respectivo detalle como lo es el valor, producto, plazo, entre otras (Más información en la hoja metadata del archivo)
obligaciones = 'Obligaciones_clientes.xlsx' 
#Archivo Tasas: tasas_productos: Contiene la tasa de interés de cada producto (Más información en la hoja metadata del archivo)
tasas = 'tasas_productos.xlsx'

#Creamos el df con las obligaciones
df_obl = pd.read_excel(obligaciones,sheet_name='Obligaciones_clientes')
#Creamos el df con las tasas
df_tas = pd.read_excel(tasas,sheet_name='Tasas')

Visualización y comprensión y ajuste de los datos

In [55]:
df_obl.head()

,radicado,num_documento,cod_segm_tasa,cod_subsegm_tasa,cal_interna_tasa,id_producto,tipo_id_producto,valor_inicial,fecha_desembolso,plazo,cod_periodicidad,periodicidad,saldo_deuda,modalidad,tipo_plazo
0,14427616502,1081648945,5,2,C4,OEL - operacion_especifica,cod_plan - producto,1.050000e+06,2023-01-31,60.0,1,MENSUAL,1.050000e+06,Nuevo,LP
1,14427616257,1032058622,2,-1,R1,0000000000097492172 - 29-Cartera Total,Obligacion - producto,2.848500e+05,2021-09-30,61.0,1,MENSUAL,2.414313e+05,Actual,LP
2,14427615471,1081374249,5,2,C4,OEL - leasing,cod_plan - producto,7.528858e+04,2023-01-31,60.0,1,MENSUAL,7.528858e+04,Nuevo,LP
3,14426546522,996136114,5,1,C1,0000000090000272791 - 29-Cartera Total,Obligacion - producto,6.240000e+05,2021-09-30,121.0,1,MENSUAL,4.375334e+05,Actual,LP
4,14427616197,1080662762,5,2,C4,0000000090000242224 - 29-Cartera Total,Obligacion - producto,1.909350e+05,2021-12-31,48.0,1,MENSUAL,1.159688e+05,Actual,LP


In [56]:
#Calculamos Nombre del producto en nueva columna

df_obl['nombre_producto'] = df_obl['id_producto'].apply(lambda x: x.rsplit('-',1)[1]).str.strip().apply(lambda x: x.split(' ',1)[0]).str.strip().str.lower()
nombre_producto=df_obl['nombre_producto'].drop_duplicates()
nombre_producto

0      operacion_especifica
1                   cartera
2                   leasing
63                  tarjeta
156                    sufi
173               factoring
238             hipotecario
Name: nombre_producto, dtype: object

In [57]:
#Visualizamos las tasas
df_tas.head()

,cod_segmento,segmento,cod_subsegmento,calificacion_riesgos,tasa_cartera,tasa_operacion_especifica,tasa_hipotecario,tasa_leasing,tasa_sufi,tasa_factoring,tasa_tarjeta
0,9,NEGOCIOS E INDEPENDIENTES,-1,C12,0.259170,0.242689,0.184613,0.146794,0.134048,0.146465,0.3867
1,9,NEGOCIOS E INDEPENDIENTES,-1,C11,0.250350,0.239824,0.206170,0.137014,0.126061,0.158310,0.3867
2,9,NEGOCIOS E INDEPENDIENTES,-1,C10,0.244242,0.237597,0.089940,0.137798,0.116774,0.164796,0.3867
3,9,NEGOCIOS E INDEPENDIENTES,-1,C9,0.232909,0.233487,0.159345,0.143824,0.123468,0.168163,0.3867
4,9,NEGOCIOS E INDEPENDIENTES,-1,C8,0.223275,0.228521,0.137500,0.144439,0.189358,0.175926,0.3867


Union de la base de obligaciones con la información de la tasa correspondiente.

In [58]:
df_obl_tas = pd.merge(df_obl,df_tas,left_on=['cod_segm_tasa','cod_subsegm_tasa','cal_interna_tasa'],right_on=['cod_segmento','cod_subsegmento','calificacion_riesgos'], how ='left')
df_obl_tas = df_obl_tas.drop(columns=['cod_segmento','cod_subsegmento','calificacion_riesgos'])
df_obl_tas.head() 

,radicado,num_documento,cod_segm_tasa,cod_subsegm_tasa,cal_interna_tasa,id_producto,tipo_id_producto,valor_inicial,fecha_desembolso,plazo,...,tipo_plazo,nombre_producto,segmento,tasa_cartera,tasa_operacion_especifica,tasa_hipotecario,tasa_leasing,tasa_sufi,tasa_factoring,tasa_tarjeta
0,14427616502,1081648945,5,2,C4,OEL - operacion_especifica,cod_plan - producto,1.050000e+06,2023-01-31,60.0,...,LP,operacion_especifica,PYME,0.190215,0.202839,0.169201,0.134434,0.111939,0.150932,0.3867
1,14427616257,1032058622,2,-1,R1,0000000000097492172 - 29-Cartera Total,Obligacion - producto,2.848500e+05,2021-09-30,61.0,...,LP,cartera,Empresarial,0.121857,0.125405,0.081032,0.082563,0.072089,0.128401,0.3867
2,14427615471,1081374249,5,2,C4,OEL - leasing,cod_plan - producto,7.528858e+04,2023-01-31,60.0,...,LP,leasing,PYME,0.190215,0.202839,0.169201,0.134434,0.111939,0.150932,0.3867
3,14426546522,996136114,5,1,C1,0000000090000272791 - 29-Cartera Total,Obligacion - producto,6.240000e+05,2021-09-30,121.0,...,LP,cartera,PYME,0.193960,0.200086,0.126630,0.127506,0.058169,0.190565,0.3867
4,14427616197,1080662762,5,2,C4,0000000090000242224 - 29-Cartera Total,Obligacion - producto,1.909350e+05,2021-12-31,48.0,...,LP,cartera,PYME,0.190215,0.202839,0.169201,0.134434,0.111939,0.150932,0.3867


In [59]:
#realizamos la asignación de la tasa segun el tipo de producto.
df_obl_tas['tasa']= df_obl_tas.apply(lambda row : row[f'tasa_{row["nombre_producto"]}'],axis=1)
df_obl_tas= df_obl_tas.drop(columns=['tasa_cartera','tasa_leasing','tasa_operacion_especifica','tasa_hipotecario','tasa_sufi','tasa_factoring','tasa_tarjeta'])
df_obl_tas

,radicado,num_documento,cod_segm_tasa,cod_subsegm_tasa,cal_interna_tasa,id_producto,tipo_id_producto,valor_inicial,fecha_desembolso,plazo,cod_periodicidad,periodicidad,saldo_deuda,modalidad,tipo_plazo,nombre_producto,segmento,tasa
0,14427616502,1081648945,5,2,C4,OEL - operacion_especifica,cod_plan - producto,1.050000e+06,2023-01-31,60.0,1,MENSUAL,1.050000e+06,Nuevo,LP,operacion_especifica,PYME,0.202839
1,14427616257,1032058622,2,-1,R1,0000000000097492172 - 29-Cartera Total,Obligacion - producto,2.848500e+05,2021-09-30,61.0,1,MENSUAL,2.414313e+05,Actual,LP,cartera,Empresarial,0.121857
2,14427615471,1081374249,5,2,C4,OEL - leasing,cod_plan - producto,7.528858e+04,2023-01-31,60.0,1,MENSUAL,7.528858e+04,Nuevo,LP,leasing,PYME,0.134434
3,14426546522,996136114,5,1,C1,0000000090000272791 - 29-Cartera Total,Obligacion - producto,6.240000e+05,2021-09-30,121.0,1,MENSUAL,4.375334e+05,Actual,LP,cartera,PYME,0.193960
4,14427616197,1080662762,5,2,C4,0000000090000242224 - 29-Cartera Total,Obligacion - producto,1.909350e+05,2021-12-31,48.0,1,MENSUAL,1.159688e+05,Actual,LP,cartera,PYME,0.190215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,14427616294,109325188,9,-1,C14,OPE - operacion_especifica,cod_plan - producto,6.750000e+04,2021-11-25,36.0,6,SEMESTRAL,4.499868e+04,Actual,LP,operacion_especifica,NEGOCIOS E INDEPENDIENTES,0.257292
2044,14427615890,960246199,5,4,C16,RLP - cartera,cod_plan - producto,5.325000e+05,2022-07-28,12.0,1,MENSUAL,2.662500e+05,Actual,CP,cartera,PYME,0.279129
2045,14427615829,1080515925,5,3,C12,OPE - operacion_especifica,cod_plan - producto,6.586189e+04,2022-02-04,12.0,1,MENSUAL,5.485251e+03,Actual,LP,operacion_especifica,PYME,0.209583
2046,14427616509,1080240535,5,2,C8,OPE - operacion_especifica,cod_plan - producto,6.000000e+04,2022-02-03,12.0,1,MENSUAL,4.885065e+03,Actual,LP,operacion_especifica,PYME,0.210004


In [60]:
#se definen variables para calcular la tasa efectiva
periodicidad = df_obl_tas['cod_periodicidad']
n = 12 / df_obl_tas['cod_periodicidad']
tasa = df_obl_tas['tasa']

#Se calcula la tasa efectiva segun formula definida 
df_obl_tas['tasa_efectiva'] = ((((1 + tasa)**(1/n))-1)*n)/n

#Se determina el Valor Final
df_obl_tas['valor_final'] =  df_obl_tas['tasa_efectiva']  * df_obl_tas['valor_inicial'] 


In [61]:
#calculamos el valor de los productos de los clientes y dejamos solo los que tienen 2 o mas 

#Valor final por cliente
#sum_por_cliente = df_obl_tas.groupby('num_documento')['valor_final'].sum()
cantidad_por_cliente = df_obl_tas.groupby(['num_documento','radicado']).size().reset_index(name = 'cantidad')

#calculamos los clientes que tienen 2 o mas
clientes_dos_mas = cantidad_por_cliente[cantidad_por_cliente['cantidad']>=2][['num_documento','radicado']]
clientes_dos_mas = clientes_dos_mas[['num_documento']]

#dejamos de la tabla de obligaciones de clientes solo los que tienen 2 o mas
df_filtrada = pd.merge(df_obl_tas,clientes_dos_mas,on=['num_documento'], how ='inner')


df_total_cliente = df_filtrada.groupby('num_documento')['valor_final'].sum().reset_index(name='valor_total')
df_total_cliente

,num_documento,valor_total
0,3907738,6741.584647
1,4234923,18035.643930
2,4935132,12880.579507
3,7061608,13730.266357
4,7099543,10100.998957
...,...,...
260,1339938146,3017.568190
261,1342616595,5636.059256
262,1344463827,11110.864288
263,1346212202,69440.932083


In [62]:
# Guarda los DataFrames como archivos pickle
df_obl_tas.to_pickle('df_obl_tas.pkl')
df_total_cliente.to_pickle('df_total_cliente.pkl')